In [42]:
from collections import Counter
import operator
import re
import psycopg2 as pg2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, probplot
import pylab
import statsmodels.stats.api as sms

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize.treebank import TreebankWordDetokenizer
#import nlp

In [3]:
conn = pg2.connect(dbname='dsjob', host='localhost',
                    password='docker', user='postgres')
conn.autocommit = True
cur = conn.cursor()

cur.execute("SELECT * FROM indeed;")
l=[]
for item in cur.fetchall():
    l.append(item)

df=pd.DataFrame(l, columns=["job_jk", "job_title",  "job_location",  "company_name", "salary", "description"])

In [4]:
df.head()

,job_jk,job_title,job_location,company_name,salary,description
0,1bce25f0b7e3723d,"Senior Data Scientist – Image Analytics, Novar...","Cambridge, MA",Novartis,No salary given,\n\nWe believe the answers are found when curi...
1,1486880d275ce7c1,Senior Scientist Deep Data Analytics,"Cambridge, MA 02142",Sanofi,No salary given,Sanofi is a global life sciences company commi...
2,7d6ac302ac846400,Senior Manager of Data Science – Cyber Security,"Boston, MA 02298",No company info,No salary given,\nPrimary Responsibilities:\n Product ownershi...
3,3100f3d85650c469,Enterprise Architect Data,"Boston, MA",MassMutual,No salary given,\nA career with us means you will work alongsi...
4,3de16480e89bcaa2,Post Doctoral Fellow,"Cambridge, MA",Eisai,No salary given,"\n\nAt G2D2, we focus not only on how such dat..."


In [5]:
vectorizer = TfidfVectorizer(stop_words = "english", max_features = None)
X = vectorizer.fit_transform(df['description'])
nmf = NMF(n_components=10, random_state =0)
W= nmf.fit_transform(X)
H = nmf.components_

In [6]:
W.shape, H.shape

((14473, 10), (10, 48220))

In [7]:
vocab = vectorizer.get_feature_names()
words_per_topic = []
for i in range(H.shape[0]):
    l=[]
    for e in H[i].argsort()[::-1][0:10]:
        l.append(vocab[e])
    print(l)
    words_per_topic.append(l)
#words_per_topic    

['learning', 'machine', 'data', 'models', 'algorithms', 'ai', 'ml', 'experience', 'science', 'deep']
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'digital', 'teams', 'experience']
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
['software', 'experience', 'cloud', 'systems', 'aws', 'development', 'design', 'technical', 'data', 'technologies']
['accenture', 'cloud', 'data', 'client', 'azure', 'visa', 'clients', 'delivery', 'gcp', 'employment']
['deloitte', 'clients', 'people', 'cognitive', 'business', 'client', 'tips', 'learn', 'help', 'offer']
['iqvia', 'healthcare', 'takes', 'clinical', 'reasonable', 'accommodation', 'patients', 'clients', 'health', 'client']
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']
['status', 'employment', 'microsoft', 'protected', 'qualified', 'applicants', 'disability', 'gender', 'information', 'national']
[

In [8]:
def max_feature_topic_search(input_pdseries,m_feature_n, topic_n):
    for m in m_feature_n:
        for n in topic_n:
            vectorizer = TfidfVectorizer(stop_words = "english", max_features = m)
            X = vectorizer.fit_transform(input_pdseries)
            nmf = NMF(n_components=n, random_state =0)
            W= nmf.fit_transform(X)
            H = nmf.components_
            vocab = vectorizer.get_feature_names()
            words_per_topic = []
            print ("max features = {}, n of topics ={}".format(m, n))
            for i in range(H.shape[0]):
                l=[]
                for e in H[i].argsort()[::-1][0:10]:
                    l.append(vocab[e])
                print("topic {}".format(i))
                print(l)
                words_per_topic.append(l)
    

In [26]:
m_feature_n =[20000, 10000, 5000, 2000, 1000]
topic_n = [40, 20, 10, 8, 6, 4]

max_feature_topic_search(df['description'], m_feature_n, topic_n)

max features = 20000, n of topics =40
topic 0
['reporting', 'bi', 'business', 'data', 'reports', 'analysis', 'ability', 'dashboards', 'tableau', 'sql']
topic 1
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'application', 'workday_recruiting', 'brave', 'arrange', 'ultimately']
topic 2
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'school', 'date']
topic 3
['software', 'systems', 'development', 'engineering', 'engineers', 'test', 'code', 'engineer', 'technical', 'infrastructure']
topic 4
['accenture', 'visa', 'client', 'clients', 'employment', 'authorization', 'status', 'delivery', 'technology', 'business']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'tips', 'learn', 'help', 'confident', 'offer', 'client']
topic 6
['learning', 'machine', 'ai', 'ml', 'algorithms', 'deep', 'models', 'science', 'nlp', 'tensorflow']
topic 7
['intelligence', 'security', 'll', 'clearance', 'warfighters', 'abroad', 'information',

max features = 20000, n of topics =8
topic 0
['data', 'business', 'analytics', 'experience', 'analysis', 'models', 'statistical', 'modeling', 'science', 'tools']
topic 1
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'strategy', 'teams', 'management']
topic 2
['research', 'university', 'teaching', 'faculty', 'health', 'position', 'department', 'students', 'information', 'program']
topic 3
['experience', 'software', 'cloud', 'learning', 'aws', 'amazon', 'systems', 'machine', 'engineering', 'development']
topic 4
['accenture', 'cloud', 'data', 'azure', 'client', 'visa', 'employment', 'delivery', 'clients', 'status']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'client', 'learn', 'business', 'tips', 'help', 'offer']
topic 6
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'clients', 'health', 'client']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']


max features = 10000, n of topics =10
topic 0
['business', 'management', 'ability', 'project', 'analytics', 'skills', 'experience', 'financial', 'analysis', 'work']
topic 1
['product', 'marketing', 'team', 'customer', 'sales', 'products', 'customers', 'business', 'work', 'teams']
topic 2
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
topic 3
['software', 'experience', 'cloud', 'systems', 'development', 'aws', 'design', 'technical', 'services', 'security']
topic 4
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'sales', 'health', 'clients']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'learn', 'tips', 'client', 'help', 'offer', 'confident']
topic 6
['learning', 'machine', 'models', 'algorithms', 'experience', 'ml', 'ai', 'techniques', 'statistical', 'data']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'information', 'dat

max features = 5000, n of topics =20
topic 0
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'reporting', 'risk', 'amp']
topic 1
['team', 'work', 'll', 'company', 'people', 'help', 'world', 'best', 'make', 'insurance']
topic 2
['university', 'research', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'position']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'technical', 'computer', 'web']
topic 4
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'clients', 'application', 'workday_recruiting', 'brave', 'client']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'tips', 'client', 'learn', 'help', 'confident', 'offer']
topic 6
['learning', 'machine', 'models', 'algorithms', 'techniques', 'statistical', 'experience', 'data', 'ml', 'ai']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'change', 'sci']
to

max features = 2000, n of topics =20
topic 0
['data', 'experience', 'analytics', 'sql', 'science', 'sets', 'tools', 'big', 'sources', 'analysis']
topic 1
['university', 'teaching', 'faculty', 'students', 'research', 'department', 'edu', 'undergraduate', 'graduate', 'college']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'best', 'world', 'make', 'build']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'computer', 'web', 'technical']
topic 4
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'risk', 'reporting', 'amp']
topic 5
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'patients', 'application', 'clients', 'workday_recruiting', 'brave']
topic 6
['status', 'employment', 'protected', 'gender', 'qualified', 'microsoft', 'disability', 'applicants', 'origin', 'race']
topic 7
['deloitte', 'clients', 'people', 'cognitive', 'client', 'tips', 'learn', 'help', 'confident', 'offer']

max features = 1000, n of topics =20
topic 0
['data', 'big', 'science', 'pipelines', 'sql', 'analytics', 'etl', 'tools', 'sets', 'sources']
topic 1
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'graduate', 'undergraduate', 'college', 'position']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'world', 'best', 'make', 'love']
topic 3
['software', 'experience', 'development', 'design', 'systems', 'engineering', 'code', 'test', 'web', 'testing']
topic 4
['learning', 'machine', 'ai', 'models', 'ml', 'algorithms', 'deep', 'science', 'data', 'techniques']
topic 5
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'application', 'patients', 'clients', 'impact', 'disabled']
topic 6
['status', 'employment', 'gender', 'protected', 'disability', 'qualified', 'applicants', 'origin', 'race', 'religion']
topic 7
['deloitte', 'clients', 'people', 'client', 'learn', 'help', 'offer', 'professionals', 'culture', 'communities']
topic 8
['statistical', 'an

In [10]:
token = nltk.word_tokenize(" ".join(df['description'].map(str)).lower())



In [11]:
len(token)

9170386

In [12]:
lemmatizer = WordNetLemmatizer()

description_lemma = [lemmatizer.lemmatize(e) for e in token]

In [13]:
len(description_lemma)
description_lemma[0:100]

['we',
 'believe',
 'the',
 'answer',
 'are',
 'found',
 'when',
 'curious',
 ',',
 'courageous',
 'and',
 'collaborative',
 'people',
 'like',
 'you',
 'are',
 'brought',
 'together',
 'in',
 'an',
 'inspiring',
 'environment',
 '.',
 'where',
 'you',
 '’',
 're',
 'given',
 'opportunity',
 'to',
 'explore',
 'the',
 'power',
 'of',
 'digital',
 'and',
 'data',
 '.',
 'where',
 'you',
 '’',
 're',
 'empowered',
 'to',
 'risk',
 'failure',
 'by',
 'taking',
 'smart',
 'risk',
 ',',
 'and',
 'where',
 'you',
 '’',
 're',
 'surrounded',
 'by',
 'people',
 'who',
 'share',
 'your',
 'determination',
 'to',
 'tackle',
 'the',
 'world',
 '’',
 's',
 'toughest',
 'medical',
 'challenge',
 '.',
 'we',
 'are',
 'novartis',
 '.',
 'join',
 'u',
 'and',
 'help',
 'u',
 're-imagine',
 'medicine',
 '.',
 'two',
 'company',
 'and',
 'one',
 'incredible',
 'alliance',
 '.',
 'novartis',
 'and',
 'microsoft',
 'have',
 'formed',
 'alliance',
 'to',
 'leverage']

In [14]:
def tokenize_lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    token = nltk.word_tokenize(" ".join(df['description'].map(str)).lower())
    return [lemmatizer.lemmatize(e) for e in token]
    

In [15]:
len(set(description_lemma))

74435

In [16]:
vocab_lem = set(description_lemma)

In [17]:
my_stop_words = stopwords.words('english')
len(my_stop_words)


179

In [18]:
my_stop_words.extend(list(string.punctuation))
my_stop_words.extend(['--'])
len(my_stop_words)

212

In [19]:
def n_grams_freq(tokens, n, stopwords):
    n_grams = ngrams(tokens,n)
    n_grams_sort = sorted(Counter(n_grams).items(), key = operator.itemgetter(1), reverse = True)
    
    return ( [(k,v) for k, v in n_grams_sort if all([e not in stopwords for e in k])] )

In [20]:
ngrams_2 = n_grams_freq(description_lemma,2, my_stop_words)
ngrams_3 = n_grams_freq(description_lemma,3, my_stop_words)
ngrams_4 = n_grams_freq(description_lemma,4, my_stop_words)

In [21]:
len(ngrams_2)

453439

In [22]:
len(ngrams_3)

469593

In [23]:
len(ngrams_4)

267015

In [24]:
ngrams_4[0:100]

[(('equal', 'opportunity/affirmative', 'action', 'employer'), 525),
 (('client', 'drive', 'healthcare', 'forward'), 416),
 (('inc.', 'provides', 'reasonable', 'accommodation'), 379),
 (('human', 'health', 'take', 'insight'), 373),
 (('best', 'work', 'every', 'day'), 319),
 (('san', 'francisco', 'fair', 'chance'), 314),
 (('francisco', 'fair', 'chance', 'ordinance'), 314),
 (('equal', 'opportunity', 'employer', '–'), 298),
 (('equal', 'opportunity-affirmative', 'action', 'employer'), 295),
 (('opportunity', 'employer', '–', 'minority'), 283),
 (('opportunity-affirmative', 'action', 'employer', '–'), 277),
 (('action', 'employer', '–', 'minority'), 271),
 (('protected', 'characteristic—to', 'fearlessly', 'drive'), 252),
 (('characteristic—to', 'fearlessly', 'drive', 'change'), 252),
 (('also', 'consider', 'qualified', 'applicant'), 248),
 (('provides', 'equal', 'employment', 'opportunity'), 244),
 (('consider', 'qualified', 'applicant', 'regardless'), 239),
 (('equal', 'employment', 'opp

In [25]:
ngrams_3[0:150]

[(('equal', 'opportunity', 'employer'), 4046),
 (('employment', 'without', 'regard'), 2319),
 (('bachelor', "'s", 'degree'), 1788),
 (('protected', 'veteran', 'status'), 1646),
 (('equal', 'employment', 'opportunity'), 1573),
 (('data', 'science', 'team'), 1550),
 (('written', 'communication', 'skill'), 1400),
 (('large', 'data', 'set'), 1014),
 (('natural', 'language', 'processing'), 935),
 (('year', '’', 'experience'), 921),
 (('machine', 'learning', 'model'), 886),
 (('verbal', 'communication', 'skill'), 879),
 (('subject', 'matter', 'expert'), 861),
 (('master', "'s", 'degree'), 834),
 (('machine', 'learning', 'algorithm'), 753),
 (('machine', 'learning', 'technique'), 743),
 (('proven', 'track', 'record'), 677),
 (('opportunity/affirmative', 'action', 'employer'), 594),
 (('affirmative', 'action', 'employer'), 590),
 (('equal', 'opportunity/affirmative', 'action'), 571),
 (('excellent', 'communication', 'skill'), 558),
 (('employer', '–', 'minority'), 556),
 (('big', 'data', 'tech

In [26]:
ngrams_2[0:1000]

[(('data', 'science'), 19085),
 (('machine', 'learning'), 12113),
 (('computer', 'science'), 7092),
 (('data', 'scientist'), 6628),
 (('sexual', 'orientation'), 6062),
 (('national', 'origin'), 5497),
 (('communication', 'skill'), 5262),
 (('gender', 'identity'), 5166),
 (('equal', 'opportunity'), 5126),
 (('big', 'data'), 4895),
 (('opportunity', 'employer'), 4554),
 (('veteran', 'status'), 4527),
 (('experience', 'working'), 4112),
 (('without', 'regard'), 3538),
 (('data', 'analysis'), 3459),
 (('related', 'field'), 3251),
 (('qualified', 'applicant'), 3197),
 (('data', 'set'), 3191),
 (('team', 'member'), 3125),
 (('best', 'practice'), 3093),
 (('bachelor', '’'), 2921),
 (('data', 'analytics'), 2693),
 (("'s", 'degree'), 2677),
 (('software', 'development'), 2602),
 (('data', 'management'), 2453),
 (('work', 'experience'), 2389),
 (('marital', 'status'), 2384),
 (('employment', 'without'), 2351),
 (('reasonable', 'accommodation'), 2323),
 (('data', 'visualization'), 2272),
 (('busi

In [27]:
ngrams_3[:100]

[(('equal', 'opportunity', 'employer'), 4046),
 (('employment', 'without', 'regard'), 2319),
 (('bachelor', "'s", 'degree'), 1788),
 (('protected', 'veteran', 'status'), 1646),
 (('equal', 'employment', 'opportunity'), 1573),
 (('data', 'science', 'team'), 1550),
 (('written', 'communication', 'skill'), 1400),
 (('large', 'data', 'set'), 1014),
 (('natural', 'language', 'processing'), 935),
 (('year', '’', 'experience'), 921),
 (('machine', 'learning', 'model'), 886),
 (('verbal', 'communication', 'skill'), 879),
 (('subject', 'matter', 'expert'), 861),
 (('master', "'s", 'degree'), 834),
 (('machine', 'learning', 'algorithm'), 753),
 (('machine', 'learning', 'technique'), 743),
 (('proven', 'track', 'record'), 677),
 (('opportunity/affirmative', 'action', 'employer'), 594),
 (('affirmative', 'action', 'employer'), 590),
 (('equal', 'opportunity/affirmative', 'action'), 571),
 (('excellent', 'communication', 'skill'), 558),
 (('employer', '–', 'minority'), 556),
 (('big', 'data', 'tech

In [28]:
description_lemma

['we',
 'believe',
 'the',
 'answer',
 'are',
 'found',
 'when',
 'curious',
 ',',
 'courageous',
 'and',
 'collaborative',
 'people',
 'like',
 'you',
 'are',
 'brought',
 'together',
 'in',
 'an',
 'inspiring',
 'environment',
 '.',
 'where',
 'you',
 '’',
 're',
 'given',
 'opportunity',
 'to',
 'explore',
 'the',
 'power',
 'of',
 'digital',
 'and',
 'data',
 '.',
 'where',
 'you',
 '’',
 're',
 'empowered',
 'to',
 'risk',
 'failure',
 'by',
 'taking',
 'smart',
 'risk',
 ',',
 'and',
 'where',
 'you',
 '’',
 're',
 'surrounded',
 'by',
 'people',
 'who',
 'share',
 'your',
 'determination',
 'to',
 'tackle',
 'the',
 'world',
 '’',
 's',
 'toughest',
 'medical',
 'challenge',
 '.',
 'we',
 'are',
 'novartis',
 '.',
 'join',
 'u',
 'and',
 'help',
 'u',
 're-imagine',
 'medicine',
 '.',
 'two',
 'company',
 'and',
 'one',
 'incredible',
 'alliance',
 '.',
 'novartis',
 'and',
 'microsoft',
 'have',
 'formed',
 'alliance',
 'to',
 'leverage',
 'data',
 '&',
 'amp',
 ';',
 'artifici

In [29]:
def get_n_gram_hyphenated(input_str, ngram_list):
    rep = {" ".join(t) : "-".join(t) for t, c in ngram_list }
    rep = dict((re.escape(k), v) for k, v in rep.items()) 
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], str(input_str).lower())

In [31]:
df_descr = df['description']
df_rep = df_descr.apply(lambda row: get_n_gram_hyphenated(row, ngrams_3[0:500]))
df_rep2 = df_rep.apply(lambda row: get_n_gram_hyphenated(row, ngrams_2[0:1500]))

In [32]:
df_rep[100]

"the tableau specialist will work closely with our internal teams and external clients in the creation, launch, maintenance, and enhancement of tableau dashboards. the goal is to aide internal and client-side analysts in deriving marketing insights from data through on-demand, live dashboards.\nresponsibilities include:\nfull lifecycle design and develop of tableau reports and dashboards\ncollaborate closely with in-house development and database teams on data-driven product solutions\ntrain and educate co-workers on tableau tools and best practices\nlead enhancement and product development discussions to continually innovate reporting and analytics\ncoordinate multiple client requests and project activities at any one time to ensure accurate, timely and efficient reporting and analysis\ncreate custom dashboards as needed based on client requests and needs\nuser license and access management\nparticipate in the design and delivery of curriculum related to the rollout of new reports and

In [33]:
vectorizer = TfidfVectorizer()

docs_tfidf = vectorizer.fit_transform(df_rep2)

In [34]:


def get_tf_idf_query_similarity(vectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: cosine similarity between query and all docs
    """
    #query_lower= " ".join(tokenize_lemmatize(query.lower()))
    #query= " ".join(tokenize_lemmatize(query))
    #query_tfidf = vectorizer.transform([query_lower])
    print(query)
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    return cosineSimilarities


In [59]:

def get_tf_idf_query_similarity2(vectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: cosine similarity between query and all docs
    """
    query = get_n_gram_hyphenated(query.lower(), ngrams_3[0:500])
    query =  get_n_gram_hyphenated(query, ngrams_2[0:1500])
    tokens = tokenize_lemmatize(query)
    #query_tfidf = vectorizer.transform([query_lower])
    query = TreebankWordDetokenizer().detokenize(tokens)
    print(str(query))
    query_tfidf = vectorizer.transform([str(query)])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    return cosineSimilarities

In [36]:
test_resume_data_scientist = '''Malik Rabb

Seattle, WA • (123) 456-7891
mrabb@email.com
SUMMARY

Data Scientist with strong math background and 3+ years of experience using predictive modeling, data processing, and data mining algorithms to solve challenging business problems. Involved in Python open source community and passionate about deep reinforcement learning.
EDUCATION
Coral Springs University
Current - Current

Bachelor of Science in Mathematics
EXPERIENCE
River Tech, Data Scientist
Current - Current

    Built fuzzy matching algorithm using k-nearest neighbors to identify non-exact matching duplicates
    Designed and developed real time recommendation engine to rank sales leads for upsell opportunities
    Refined personalization algorithms for 1M+ customers on web and mobile
    Transformed raw data into MySQL with custom-made ETL application to prepare unruly data for machine learning

Retail Ocean, Data Scientist
Current - Current

    Leveraged 200M+ tweets to develop sentiment analysis model that helped improve sales and marketing strategies
    Used Python and Spark to scrape, clean, and analyze large datasets
    Helped build tools for detecting botnets with machine learning and data mining

SKILLS

    2nd place at Coral Springs Big Data Hackathon (out of 150+ participants)
    Java, Python, C++, Hadoop ecosystem, and MySQL
    Data cleansing, modeling, and mining
    Machine learning
'''

In [37]:
test_resume_software_enginner = '''Ivy Haddington

Austin, TX • (123) 456-7891
ihaddington@email.com
SUMMARY

Focused and quick-learning Software Engineer with 3 years of experience in computer science, programming, and UX design for various projects and clients.
EDUCATION
Hawaii Western
Aug '10 - May '14

Bachelor of Science in Electrical Engineering
EXPERIENCE
Crane & Jenkins, Software Engineer
Feb '15 - Current

    Integrate code changes, test code changes, and document code changes
    Developed Perl and PHP scripts to translate data between applications
    Developed automation scripts to test storage appliances in Python
    Practice agile development methodologies and work with current networking technologies
    Used Hadoop to build a scalable distributed data solution

Tradelot, Software Developer
Current - Current

    Wrote, modified, and debugged software for clients
    Developed code to complete assigned project by deadline
    Developed HTML, CSS, JavaScript, and JSP pages for user interaction
    Wrote Python scripts to load data and parse XML documents 

SKILLS

    Certified Scrum Master
    Cisco Certified Network Associate
    Cisco Certified Network Professional Security
    Unix and Linux
    Oracle
    SQL
    Java
    C++
    HTML
    '''

In [52]:
test_resume_accountant = '''


Marie Anderson
Sometown, CO 55555 | 555-555-5555 | ma@somedomain.com | LinkedIn URL

STAFF ACCOUNTANT
Accounting and auditing | Corporate and personal taxes | Management consulting

Build long-term client relationships through “hands-on” approach and commitment to service

    Diligent accountant backed by public accounting, general ledger (GL) and tax experience and expertise in GAAP, data analytics and sophisticated modeling tools.
    Strategic business partner delivering fiscally responsible strategies to resolve financial and accounting challenges, propel corporate growth and strengthen compliance.
    Highly technicalexperienced user of QuickBooks, Drake, Microsoft Dynamics GP, JD Edwards EnterpriseOne, Hyperion, SAP, Oracle, IBM Cognos, Visual Basic, SQL and Excel.

Skills:

    Corporate Accounting
    General Ledger
    GAAP Standards
    SOX Compliance

    Financial Reporting & Analysis
    Financial Statements
    Forecasting & Projections
    M&A Due Diligence

    Tax Preparation
    Audit Reviews
    Financial Research
    State & Federal Tax Codes

Experience

ABC COMPANY (Sometown, CO) Mid-size public accounting firm
Accountant, 2012 to Present
Accounting Intern, 2011 to 2012

Provide professional accounting services for individuals, businesses and government clients, from tax preparation to audit support, financial statement preparation, pro forma budgeting, GL accounting and bank reconciliation.
 

Consistently deliver the highest quality, full-spectrum accounting services. Success examples:

    Selected for position as reviewer in ABC Co’s Audit Quality Review Process (AQRP) from 2015 through 2016. Assessed quality of audits completed by other partners and reported findings to national office.
    Leveraged knowledge of federal tax code to win IRS abatements that averted tax penalties of $11,500 and $7,000 for two farm operations.
    Implemented systems for expanded accounting, reporting and stock option transaction activities to support tech startup during period of fast growth (revenues increased from $500K in 2012 to $12M+ in 2016).
    Helped businesses accelerate month-end close and payroll processing by up to 25% while reducing errors and increasing accuracy to new bests.
    Assisted with SOX compliance by ensuring on-time delivery of supporting documentation to auditors.
    Led financial modeling and due diligence to evaluate proposed $5M acquisition for real estate development company. Uncovered significant GL discrepancies, winning client praise for preventing unwise investment.

Previous Experience: Bookkeeper (2008 to 2011) at DEF COMPANY (a plastics manufacturer in Sometown, CO) and Volunteer Tax Preparer through the IRS VITA program during college.

Education

XYZ UNIVERSITY (Sometown, CO)
Bachelor of Science in Accounting, Minor in Business

Preparing to sit for State of Colorado CPA exam this spring
'''

In [55]:
test_resume_webdeveloper = '''


Jim March
Sometown, NY 55555 | 555-555-5555 | jm@somedomain.com | LinkedIn URL | Twitter Handle

Web Developer

Creative web developer dedicated to building and optimizing the performance of user-centric, high-impact websites for nationwide, F500 and global companies. Leverage technical, analytical and problem-solving skills to create dynamic, high-speed websites, apps and platforms fueling competitive advantage and revenue growth.

Expertise

    Web Development
    Coding & Programming
    Search Engine Optimization
    Cross-Browser Compatibility

    Site Troubleshooting
    UX & UI Design
    Agile Methodologies
    Hosting & Maintenance

    Digital & eCommerce Strategy
    CRM & CMS Platforms
    Information Architecture
    Multimedia Design

Professional Experience

ABC AGENCY, INC. - Sometown, NY • Web Developer, 2013 to Present

Work collaboratively with clients and in-house agency teams to provide rapid, robust and client-acclaimed front- and back-end web development optimizing user experience, search engine ranking, sales, brand positioning and related metrics.

Representative Projects & Results:

    Global Cosmetics Company: Developed API platform for segmentation, personalized recommendations and omni-channel messaging that reduced cart-abandonment rate by 37%, leading to a $1.25M increase in online sales within 90 days of solution launch.
    Pharmaceutical Manufacturer: Led front-end development and assisted with back-end programming for website overhaul. Ensured the technical feasibility and optimum functioning of design features that accelerated load-time by 17% and improved site stickiness by 21%.
    NYC Real Estate Firm: Increased web traffic 125% by improving navigation, creating dynamic media sections, introducing virtual tours and adding social media plugins.
    Luxury Resort Hotel Group: Developed SEO strategy as well as seamless interface with travel booking sites that resulted in top 5 rankings on Google, Bing and Yahoo! coupled with a 22% increase in annual sales from Expedia, Hotels.com, Hotwire and Trivago.
    Restaurant Chain: Contributed to mobile app development and resolved issues causing site crashes, elevating uptime to 99%. Efforts were key to a 27% increase in eCommerce sales.

 

DEF AGENCY, INC. — Sometown, NY • WordPress Web Developer Intern, 2012 to 2013

Worked with award-winning developers to create and update WordPress sites. Optimized sites for usability, search engine optimization and accessibility. Partnered with marketing department to continually perfect client pitches. Offered job at conclusion of internship (declined to pursue web developer opportunity at ABC Agency).

Education

SOMETOWN UNIVERSITY • Bachelor of Applied Science in Internet and Web Development

Technology Summary (complete list on request)

HTML5; XHTML; CSS3; JavaScript; jQuery; PHP; SQL; Bootstrap; .NET; ActionScript; Adobe Creative Cloud; WordPress; Google Analytics; AJAX; CMS Tools; ColdFusion; Windows; Mac; iOS; Android
'''

In [61]:
test_resume_db_admin='''
Ann Simpson
Sometown, NY 10000 | H: 718-555-5555 | C: 917-555-5555 | as@somedomain.com | LinkedIn URL

Database Administrator

Database administrator (DBA) with extensive experience designing, coding, testing and supporting next-generation database solutions in Oracle enterprise and SQL Server environments. Proficient in an assortment of technologies, including Oracle 12c, DB2, Access, Sybase, MS SQL Server 2016, JDBC, Visio, Apache HTTP Server, Java, C++, XML, Windows, Linux and Unix. DBA skills include:

    Oracle Database Administration
    Performance Tuning & Capacity Monitoring
    Data Security, Backup & Recovery
    Standby/Failover Administration

	

    Oracle RAC & ASM Administration/Installation
    Oracle Forms and Reports Development
    Database-Backed Web Solutions
    SQL Tuning in an Oracle Environment

Experience

ABC COMPANY, Sometown, NY | 5/2010 to Present

Database Administrator

Manage Oracle database administration assignments for diverse industry clients. Enhance database performance, perform restores, implement recovery procedures, handle performance tuning and conduct regular system backups. Ensure technical and functional designs meet business requirements.

Key projects:

    Served as DBA or project lead in the completion of 80+ medium- to large-scale implementations, managing projects from business requirements analysis to solutions delivery and support.
    Managed a $1.2 million data-integration project for financial services firm that consolidated information from accounting applications, third-party market data and internal equities/fixed income applications.
    Contributed to furniture retailer’s 15% revenue growth in 2016 by standardizing content from disparate databases, enabling sales and support staff to quickly respond to customer requests.
    Created Web-enabled, group-scheduling system for a large university, allowing students to view and print schedules for current and future semesters. Delivered solution on time, on spec and on budget.
    Innovated time-saving, robust employee data-intake system that automated database update functions, enabling new salary and process-exception information to be automatically populated.
    Developed and tested storage strategies and disaster-recovery plan for large manufacturing company’s operational database, delivering solution that guaranteed recovery performance and high availability.
    Enhanced users’ understanding of database systems by conducting pre-implementation workshops, delivering group and individual training sessions and creating user-friendly training materials.

DEF COMPANY, Sometown, NY | 1/2010 to 5/2010

IT Internship

Selected for competitive internship with leading consulting firm. Developed functions, scripts and reports that were instrumental to the success of high-priority development projects.

Key projects:

    Developed 25 custom financial and auditing reports using SAP Crystal Reports with T-SQL, JDE Report Design Aid, MS Excel and Access.

Education & Training

ABC UNIVERSITY, Sometown, NY | 5/2010
Bachelor of Science, Major in Computer Science
DBA Workshops: Oracle Database 12c Administration, Advanced Oracle DBA, Intro to Oracle
'''

In [38]:
def n_matching_jobs(resume_str, vectorizer, docs_tfidf, n):
    similarities = get_tf_idf_query_similarity(vectorizer, docs_tfidf, resume_str)
    return df[['job_title', 'job_location', 'description']].iloc[similarities.argsort()[::-1][0:n]]

def n_matching_jobs2(resume_str, vectorizer, docs_tfidf, n):
    similarities = get_tf_idf_query_similarity2(vectorizer, docs_tfidf, resume_str)
    return df[['job_title', 'job_location', 'description']].iloc[similarities.argsort()[::-1][0:n]]

In [341]:
n_matching_jobs(test_resume_software_enginner, vectorizer, docs_tfidf, 20)

,job_title,job_location,description
9553,Data Management Lead CI/CD,"Sterling, VA 20166","Sterling, VA 20166\n\nCulmen International is ..."
14286,Senior Applications Engineer JAVA Oracle Labs,"Redwood City, CA 94065",\n\nPreferred Qualifications\n\nJob Descriptio...
14312,Principal Applications Engineer JAVA Oracle ...,"Redwood City, CA 94065",\n\nPreferred Qualifications\n\nJob Descriptio...
10671,Software Engineer Level 3 (ID),"Annapolis Junction, MD",ProSync is seeking an experienced Software Eng...
6411,Software Developer,"Chantilly, VA 20151",\nThe Software Application Programmer will des...
8218,Software Engineer in Test,"Chicago, IL 60637",Department\n2010009 BSD - Center for Data Inte...
10666,Software Engineer Level 2/3 (ID),"Annapolis Junction, MD",ProSync is seeking an experienced Software Eng...
10660,Software Engineer Level 3 (ID),"Annapolis Junction, MD",ProSync is seeking an experienced Software Eng...
6936,Senior Software Engineer,"Ashburn, VA 20147",The Senior Software Engineer will be responsib...
7866,DevOps Technical Lead,"Seattle, WA","Reporting to the Global DevOps Practice Lead, ..."


In [342]:
n_matching_jobs(test_resume_data_scientist, vectorizer, docs_tfidf, 20)

,job_title,job_location,description
8442,Senior Data Scientist,"Milwaukee, WI",\nRole Summary:\nThe Senior Data Scientist app...
3272,Data Architect,"Raleigh, NC 27609",\nPurpose:\n\nTo turn raw data into valuable i...
12138,Data Scientist,"Plano, TX 75023",\nThe Lead Data Scientist will support the suc...
4632,Associate Data Scientist/Data Scientist,"Allentown, PA 18101","\nPosition located in Allentown, PA or Washing..."
4664,Associate Data Scientist / Data Scientist,"Allentown, PA 18101",\nWe've got an electric future. The energy ind...
6973,Manager Data Science,"Arlington, VA","Coming from a quantitative science discipline,..."
7073,Data Scientist,"Reston, VA",\nAbout the Role\n\nX-Mode is looking for a fu...
6355,Data Scientist,"Arlington, VA",AM LLC is seeking a data-scientist with analyt...
6453,Data Scientist,"Alexandria, VA",\n\nSteel Point Solutions is looking for a Dat...
6020,Principal Data Scientist,"Salt Lake, UT","Summary\n\nData analysis, modeling, and foreca..."


In [54]:
n_matching_jobs(test_resume_accountant, vectorizer, docs_tfidf, 20)




Marie Anderson
Sometown, CO 55555 | 555-555-5555 | ma@somedomain.com | LinkedIn URL

STAFF ACCOUNTANT
Accounting and auditing | Corporate and personal taxes | Management consulting

Build long-term client relationships through “hands-on” approach and commitment to service

    Diligent accountant backed by public accounting, general ledger (GL) and tax experience and expertise in GAAP, data analytics and sophisticated modeling tools.
    Strategic business partner delivering fiscally responsible strategies to resolve financial and accounting challenges, propel corporate growth and strengthen compliance.
    Highly technicalexperienced user of QuickBooks, Drake, Microsoft Dynamics GP, JD Edwards EnterpriseOne, Hyperion, SAP, Oracle, IBM Cognos, Visual Basic, SQL and Excel.

Skills:

    Corporate Accounting
    General Ledger
    GAAP Standards
    SOX Compliance

    Financial Reporting & Analysis
    Financial Statements
    Forecasting & Projections
    M&A Due Diligence

    Tax 

,job_title,job_location,description
9981,"Supervisor, Tax FPA & Modeling","Glendale, CA","\nPosition requires strong communication, inte..."
11963,Fund Accountant,"New York, NY",The Fund Accounting team’s mission is to deliv...
14010,Controller,"San Francisco, CA",Imagine if every time you went to the doctor y...
11988,Tax Manager,"New York, NY",We are seeking a tax professional with alterna...
9424,Controller,"Boston, MA 02110","More than 30,000 organizations in over 150 cou..."
2572,Tax Sr. Associate Tax Transformation Practice,"Charlotte, NC 28209",DHG ranks among the top 20 public accounting f...
8796,University Recruiting Campus Events Entry Lev...,"Austin, TX 78735","Entry Level Finance, Accounting, Audit\nNXP’s ..."
1725,Tenure Track Position in Accounting All Ranks,"Jersey City, NJ 07305",\nDEPARTMENT: ACCOUNTING\n\nPOSITION: TENURE- ...
3152,Accounting Manager,"Morrisville, NC 27560",A B2B technology and data-driven solutions com...
4709,Finance Systems Manager,"Providence, RI 02903","We’re here to create a safer, happier and more..."


In [58]:
n_matching_jobs(test_resume_webdeveloper, vectorizer, docs_tfidf, 20)




Jim March
Sometown, NY 55555 | 555-555-5555 | jm@somedomain.com | LinkedIn URL | Twitter Handle

Web Developer

Creative web developer dedicated to building and optimizing the performance of user-centric, high-impact websites for nationwide, F500 and global companies. Leverage technical, analytical and problem-solving skills to create dynamic, high-speed websites, apps and platforms fueling competitive advantage and revenue growth.

Expertise

    Web Development
    Coding & Programming
    Search Engine Optimization
    Cross-Browser Compatibility

    Site Troubleshooting
    UX & UI Design
    Agile Methodologies
    Hosting & Maintenance

    Digital & eCommerce Strategy
    CRM & CMS Platforms
    Information Architecture
    Multimedia Design

Professional Experience

ABC AGENCY, INC. - Sometown, NY • Web Developer, 2013 to Present

Work collaboratively with clients and in-house agency teams to provide rapid, robust and client-acclaimed front- and back-end web development opt

,job_title,job_location,description
2280,Web Development Team Lead,"Rochester, NY",Responsibilities:\nLead the LLE IT web strateg...
3409,Development Manager,"Columbus, OH 43215","The future of marketing is more personal, more..."
5180,System Architect Web Design/Software Development,"San Antonio, TX 78234",\n\nREQUIRED QUALIFICATIONS:\n\nTOP SECRET/SCI...
5354,Web Developer,"College Station, TX 77845",\n\nResponsibilities include the following:\nD...
14395,Web Developer,"San Francisco, CA 94704",\nThe Databricks marketing website is our sing...
1255,Mobile Application Developer,"Princeton, NJ 08540",As mobile application developer on the Covance...
1670,Mobile Application Developer,"Princeton, NJ 08540",As mobile application developer on the Covance...
6202,VICE PRESIDENT OF TECHNOLOGY,"Burlington, VT 05401",Responsibilities:\nLead our development team (...
5832,"Senior Designer, Data Visualization","Austin, TX 78701",\nThe designer should be well versed in Wordpr...
5042,Senior Business Data AnalystWeb/Digital Analytics,"Plano, TX 75023",We have an exciting opportunity on a team that...


In [62]:
n_matching_jobs(test_resume_db_admin, vectorizer, docs_tfidf,20)


Ann Simpson
Sometown, NY 10000 | H: 718-555-5555 | C: 917-555-5555 | as@somedomain.com | LinkedIn URL

Database Administrator

Database administrator (DBA) with extensive experience designing, coding, testing and supporting next-generation database solutions in Oracle enterprise and SQL Server environments. Proficient in an assortment of technologies, including Oracle 12c, DB2, Access, Sybase, MS SQL Server 2016, JDBC, Visio, Apache HTTP Server, Java, C++, XML, Windows, Linux and Unix. DBA skills include:

    Oracle Database Administration
    Performance Tuning & Capacity Monitoring
    Data Security, Backup & Recovery
    Standby/Failover Administration

	

    Oracle RAC & ASM Administration/Installation
    Oracle Forms and Reports Development
    Database-Backed Web Solutions
    SQL Tuning in an Oracle Environment

Experience

ABC COMPANY, Sometown, NY | 5/2010 to Present

Database Administrator

Manage Oracle database administration assignments for diverse industry clients. En

,job_title,job_location,description
9036,Senior Oracle DBA,"Reston, VA 20190",\n\nNo Visa Sponsorship is available for this ...
4273,DBA PostgreS,"Blue Bell, PA",\nDBA – PostgreSQL\n\nAs a Senior Database Adm...
11357,SQL Developer,"Atlanta, GA 30339","Design, build, and maintain efficient, reusabl..."
6649,Database Programmer (DBA),"Arlington, VA 22201",Responsibilities and Duties Include:\nMaintain...
9003,Senior Database Administrator PostgreS,"Washington, DC","\nAs a Senior Database Administrator, you will..."
8227,Distributed Systems Cloud Engineer,"Seattle, WA",\n\nPreferred Qualifications\n\nOCI Data Engin...
8231,Distributed Systems Cloud Engineer,"Seattle, WA",\n\nPreferred Qualifications\n\nOCI Data Engin...
1002,Database Administrator Anesthesiology,"St. Louis, MO 63130",\n\n\nThis position is full-time and works app...
5352,Database Administrator I,"Houston, TX 77002",The Database Administrator II position will re...
11046,OCI Software Development Senior Manager,"Washington, DC 20005",\n\nPreferred Qualifications\n\nJob Descriptio...


In [60]:
n_matching_jobs2(test_resume_software_enginner, vectorizer, docs_tfidf, 20)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,job_title,job_location,description
661,Data Platform Solutions Manager,"Saint Paul, MN 55104",The Manager of Data Platform Solutions is an i...
10770,"Director, Data Science Partner & Analytics, Ma...","Bethesda, MD 20817",Job Number 19166395\nJob Category Sales and Ma...
8994,"Senior Director, Data and Analytics","Rockville, MD 20850",This is a great opportunity to be a part of a ...
10384,"Director, Data Science Partner & Analysis Brand","Bethesda, MD 20817",Job Number 19173359\nJob Category Sales and Ma...
1240,"Director, Mortgage Information Architecture & ...","Iselin, NJ 08830","The Director, Mortgage Information Architectur..."
1381,"Director, Mortgage Information Architecture & ...","Iselin, NJ 08830","The Director, Mortgage Information Architectur..."
3411,"Consultant, Business Data Management (Senior D...","Columbus, OH 43215",Position Background\nThe Property and Casualty...
3873,Data Science and Analytics Team Lead,"Bend, OR 97701",We started in 1952 when Les Schwab bought one ...
2108,Transaction Advisory Services Senior Strategy...,"New York, NY",\nWe work on the most complex and high-profile...
3223,Principal Data Engineer,"Charlotte, NC 28277","\nAt Brighthouse Financial, we’re fostering a ..."


In [46]:
n_matching_jobs2(test_resume_data_scientist, vectorizer, docs_tfidf, 20)

,job_title,job_location,description
661,Data Platform Solutions Manager,"Saint Paul, MN 55104",The Manager of Data Platform Solutions is an i...
10770,"Director, Data Science Partner & Analytics, Ma...","Bethesda, MD 20817",Job Number 19166395\nJob Category Sales and Ma...
8994,"Senior Director, Data and Analytics","Rockville, MD 20850",This is a great opportunity to be a part of a ...
10384,"Director, Data Science Partner & Analysis Brand","Bethesda, MD 20817",Job Number 19173359\nJob Category Sales and Ma...
1240,"Director, Mortgage Information Architecture & ...","Iselin, NJ 08830","The Director, Mortgage Information Architectur..."
1381,"Director, Mortgage Information Architecture & ...","Iselin, NJ 08830","The Director, Mortgage Information Architectur..."
3411,"Consultant, Business Data Management (Senior D...","Columbus, OH 43215",Position Background\nThe Property and Casualty...
3873,Data Science and Analytics Team Lead,"Bend, OR 97701",We started in 1952 when Les Schwab bought one ...
2108,Transaction Advisory Services Senior Strategy...,"New York, NY",\nWe work on the most complex and high-profile...
3223,Principal Data Engineer,"Charlotte, NC 28277","\nAt Brighthouse Financial, we’re fostering a ..."


In [49]:
m_feature_n =[5000, 2000, 1000]
topic_n = [10, 8, 6, 4, 3, 2]
max_feature_topic_search(df_rep2, m_feature_n, topic_n)

max features = 5000, n of topics =10
topic 0
['business', 'management', 'ability', 'analytics', 'project', 'skills', 'experience', 'financial', 'analysis', 'work']
topic 1
['product', 'marketing', 'team', 'customer', 'sales', 'products', 'customers', 'company', 'work', 'teams']
topic 2
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'edu', 'science', 'candidates']
topic 3
['software', 'experience', 'cloud', 'systems', 'development', 'aws', 'design', 'technical', 'services', 'engineering']
topic 4
['iqvia', 'healthcare', 'takes', 'clinical', 'accommodation', 'reasonable', 'patients', 'sales', 'health', 'clients']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'client', 'learn', 'tips', 'help', 'offer', 'confident']
topic 6
['learning', 'machine', 'models', 'algorithms', 'experience', 'ml', 'statistical', 'techniques', 'ai', 'data']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'information',

max features = 2000, n of topics =2
topic 0
['data', 'experience', 'business', 'team', 'analytics', 'work', 'product', 'solutions', 'learning', 'skills']
topic 1
['research', 'university', 'health', 'information', 'position', 'teaching', 'faculty', 'required', 'program', 'employment']
max features = 1000, n of topics =10
topic 0
['data', 'analytics', 'sql', 'experience', 'business', 'tools', 'science', 'big', 'sets', 'analysis']
topic 1
['research', 'university', 'faculty', 'teaching', 'students', 'department', 'edu', 'position', 'graduate', 'science']
topic 2
['product', 'marketing', 'team', 'sales', 'customer', 'company', 'products', 'work', 'teams', 'll']
topic 3
['experience', 'software', 'cloud', 'development', 'systems', 'design', 'engineering', 'technical', 'technologies', 'services']
topic 4
['learning', 'machine', 'models', 'algorithms', 'experience', 'techniques', 'statistical', 'research', 'science', 'ai']
topic 5
['iqvia', 'healthcare', 'clinical', 'health', 'takes', 'accom